## Objectives:
### What is the impact of traffic?  
- What is the porbability of having fatal or serious accident within certain conditions of traffic? In what conditions, there will be the most likely of having fatal accidents so that the driver should be more careful to avoid that.
- As the record data of collision we have in Chicago, base on various attributes, we will build a model that can predict the severity of the accident.
In order to do that, we collect the data of observations from different accidents from years, there are some attributes we do not use such as ID, KEY, etc.     
### <font color = 'blue'>What We will do is as following:</font>
- Estimate ***null*** values and decide whether to drop them or replace them
- Cleanse the data, check the ***data types*** whether they are in correct types
- Explore the data and pick only the ***correlated attributes*** to the target, which is serverity. We will drop the others
- Make the data more intuitive by using ***visualization techniques***
- ***Build the model*** from different algorithms and evaluate each model to get the most accuracy one.
- ***Predict*** the outcome from that model from the unseen dataset

### What is CRISP - DM?
It stands for <font color = 'green'>"Cross - Industry Standard Process"</font> for <font color = 'green'>"Data Mining"</font>  
The process consists of: ***Business Understanding, Data Understanding, Data Preparation and Cleaning, Modeling, Evaluation, Deployment***  
  
1. ***Business Understanding:*** Understand the objective from the businees or application perspective
2. ***Data Understanding:*** collect and distract the data from the various sources, then determine the attributes that will be used to train the model
3. ***Data Preparation:*** construct the final dataset, can be performed multiple times and includes balancing the labeled data, transformation, filling the missing values, and cleaning the dataset
4. ***Modelling:*** select algorithms and methods to build the model. (SVM, XGBoost, decision tree, etc)
5. ***Evaluation:*** to ensure that the business or application's objectives are achieved. (accuracy, recall, f1-score, precision, confusion-matrix)
6. ***Deployment:*** it varies, could be creating the report, developing interative visualization, making a machine learning model available in the production environment

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Show the maximum columns of the dataset
pd.set_option('display.max_columns',60) 

# Remove warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv'
df = pd.read_csv(url)

In [3]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


### What do all the columns mean?
- ***SEVERITYCODE*** : A code that corresponds to the severity of the collision.
    - 3: fatality
    - 2b: serious injury
    - 2: injury
    - 1: prop damage
    - 0: Unknow
- ***OBJECTID:***         unique identifier
- ***ADDRTYPE:***         Collision address type ("Alley, Block, Intersection")
- ***X, Y:***             longitude and latitude
- ***SEVERITYDESC:***     a detailed description of the severity of the collision
- ***PEDCOUNT:***         the number of pedestrians involved in the collision
- ***PEDCYLCOUNT:***      the number of bicycle involved in the collision
- ***VEHCOUNT:****        the number of vehicle invoved in the collision
- ***INJURIES:***         The number of injuries in the collision
- ***SERIOUSINJURIES:***  the number of serious injuries in the collision
- ***FATALITIES:***       the number of the fatalities in the collision
- ***INCDATE:***          the date of the incident
- ***JUCTIONTYPE:***      category of juction at which collision took place
- ***INATTENTIONIND:***   whether or not the collision was due to the inattention
- ***UNDERINFL:***        whether or not the driver involved was under the infulence of drugs or alcohol
- ***WEATHER:***          weather condition during the time of collision
- ***ROADCOND:***         the condition of road
- ***LIGHTCOND:***        the light condition
- ***SPEEDING:***         whether or not the speeding was a factor of the collision
- ***HITPARKEDCAR:***     whether or not the collision involved hitting a park car

In [4]:
df.describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.149360e+05,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,37558.450576,1.298901,2.444427,0.037139,0.028391,1.920780,13.867768,7.972521e+06,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,51745.990273,0.457778,1.345929,0.198150,0.167413,0.631047,6.868755,2.553533e+06,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,28667.000000,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,6.040015e+06,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,29973.000000,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,8.023022e+06,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,33973.000000,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,1.015501e+07,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,219547.000000,331454.000000,332954.000000,757580.000000,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.307202e+07,525241.000000,5.239700e+06


In [5]:
print("Size of Dataset: ", df.shape)

Size of Dataset:  (194673, 38)


In [6]:
list_per = []
for i in df.columns:
    count = df[i].isnull().sum()
    percent = (count / len(df[i]) * 100).round(1)
    list_per.append(percent)
table = pd.DataFrame()
table['Column name'] = df.columns
table['Percent of missing values'] = list_per
table = table.sort_values(by = ['Percent of missing values'], ascending = False)
table.head(10)

,Column name,Percent of missing values
30,PEDROWNOTGRNT,97.6
12,EXCEPTRSNDESC,97.1
32,SPEEDING,95.2
25,INATTENTIONIND,84.7
9,INTKEY,66.6
11,EXCEPTRSNCODE,56.4
31,SDOTCOLNUM,41.0
22,JUNCTIONTYPE,3.3
29,LIGHTCOND,2.7
1,X,2.7


---
### Dealing with missing values:
- Since there a few columns have the large percentage of missing values, we merely drop them (column index : 30, 12, 32, 25, 9, 11, 31)
- For the rest of columns, as the percentage of missing is small (<5%), so that we dop these observations

In [7]:
df.drop(columns = ['PEDROWNOTGRNT', 'EXCEPTRSNDESC', 'SPEEDING', 'INATTENTIONIND', 'INTKEY', 'EXCEPTRSNCODE', 'SDOTCOLNUM'], axis = 1, inplace = True)

# We also drop unnecessary columns such as OBJECT ID, INCKEY, COLDETKEY, REPORTNO, SDOT_COLCODE, ST_COLCODE
df.drop(columns = ['OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO','SDOT_COLCODE', 'ST_COLCODE'], axis = 1, inplace = True)

In [8]:
df.dropna(inplace = True)
print('Size of dataset after dropping missing values:', df.shape)

Size of dataset after dropping missing values: (180067, 25)


In [9]:
df.head()

,SEVERITYCODE,X,Y,STATUS,ADDRTYPE,LOCATION,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLDESC,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,Matched,Intersection,5TH AVE NE AND NE 103RD ST,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",N,Overcast,Wet,Daylight,Entering at angle,0,0,N
1,1,-122.347294,47.647172,Matched,Block,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",0,Raining,Wet,Dark - Street Lights On,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,Matched,Block,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,Overcast,Dry,Daylight,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,Matched,Block,2ND AVE BETWEEN MARION ST AND MADISON ST,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",N,Clear,Dry,Daylight,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,Matched,Intersection,SWIFT AVE S AND SWIFT AV OFF RP,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,Raining,Wet,Daylight,Entering at angle,0,0,N


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180067 entries, 0 to 194672
Data columns (total 25 columns):
SEVERITYCODE      180067 non-null int64
X                 180067 non-null float64
Y                 180067 non-null float64
STATUS            180067 non-null object
ADDRTYPE          180067 non-null object
LOCATION          180067 non-null object
SEVERITYCODE.1    180067 non-null int64
SEVERITYDESC      180067 non-null object
COLLISIONTYPE     180067 non-null object
PERSONCOUNT       180067 non-null int64
PEDCOUNT          180067 non-null int64
PEDCYLCOUNT       180067 non-null int64
VEHCOUNT          180067 non-null int64
INCDATE           180067 non-null object
INCDTTM           180067 non-null object
JUNCTIONTYPE      180067 non-null object
SDOT_COLDESC      180067 non-null object
UNDERINFL         180067 non-null object
WEATHER           180067 non-null object
ROADCOND          180067 non-null object
LIGHTCOND         180067 non-null object
ST_COLDESC        180067 non-nul

In [11]:
print('Distinct values of Underinfluence',df['UNDERINFL'].unique())
print('Distince values of HITPARKEDCAR', df['HITPARKEDCAR'].unique())

Distinct values of Underinfluence ['N' '0' '1' 'Y']
Distince values of HITPARKEDCAR ['N' 'Y']


In [12]:
print('WEATHER:', df['WEATHER'].unique())
print('ROAD CONDITION',df.ROADCOND.unique())
print('LIGHT CONDITION', df.LIGHTCOND.unique())

WEATHER: ['Overcast' 'Raining' 'Clear' 'Unknown' 'Other' 'Snowing' 'Fog/Smog/Smoke'
 'Sleet/Hail/Freezing Rain' 'Blowing Sand/Dirt' 'Severe Crosswind'
 'Partly Cloudy']
ROAD CONDITION ['Wet' 'Dry' 'Unknown' 'Snow/Slush' 'Ice' 'Other' 'Sand/Mud/Dirt'
 'Standing Water' 'Oil']
LIGHT CONDITION ['Daylight' 'Dark - Street Lights On' 'Dark - No Street Lights' 'Unknown'
 'Dusk' 'Dawn' 'Dark - Street Lights Off' 'Other'
 'Dark - Unknown Lighting']


---
### Dealing with data types:
- We convert UNDERINFL into integer values by replacing 'N' as 0 and 'Y' as 1
- We convert HITPARKEDCAR into binary by replacing 'N' as 0 and 'Y' as 1

In [13]:
df['UNDERINFL'].replace(to_replace = ['N','Y'], value =[0, 1], inplace = True)
df['UNDERINFL'] = df['UNDERINFL'].astype(int)

df['HITPARKEDCAR'].replace(to_replace = ['N','Y'], value =[0, 1], inplace = True)
df['HITPARKEDCAR'] = df['HITPARKEDCAR'].astype(int)